In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
import abc
import torch

import proteinbert_gen.constants as consts

from proteinbert_gen.word_freq import create_word_freq_tensor

In [1]:
def word_freq_preprocess_fn(wf):
    wf = wf + 1
    wf = wf.log()
    wf = wf / wf.max()

    # range: 0 - 1
    return wf

def process_fn_in_collate(wf):
    return wf - wf.mean()

In [10]:
wf_tensor = create_word_freq_tensor("../data/sprot_1m_word_freq_dict.pkl", consts.ALL_TOKENS)
wf_tensor = word_freq_preprocess_fn(wf_tensor)

In [11]:
wf_tensor

tensor([0.9930, 0.8761, 0.9626, 0.9751, 0.9425, 0.9825, 0.9083, 0.9710, 0.9673,
        1.0000, 0.9151, 0.9419, 0.9519, 0.9393, 0.9653, 0.9709, 0.9612, 0.3391,
        0.9802, 0.8617, 0.5234, 0.9241, 0.0000, 0.0000, 0.0000, 0.0000])

In [9]:
class SampleClassBase(abc.ABC):
    def sample(self, logits, x_0):
        raise NotImplementedError

    def post_process_sample_in_prediction(self, sample, x_0):
        return sample


class Categorical(SampleClassBase):
    def sample(self, logits, x_0):
        return torch.distributions.categorical.Categorical(logits=logits).sample()